<a href="https://colab.research.google.com/github/ChandrakantPrajapat/Google-Colab/blob/main/OptiNav.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
from IPython.display import display
import ipywidgets as widgets

import pandas as pd
import matplotlib.pyplot as plt

# %% [markdown]
# Upload your CSV files. You can select multiple files.

# %%
uploaded = files.upload()

# Get filenames and sort them alphabetically
filenames = sorted(uploaded.keys())

# Dictionary to store DataFrames, with filenames as keys
dataframes = {}

# Store the current file index
current_file_index = 0

# Input for CPI (assuming it's the same for all files or you'll enter it for each)
# For simplicity, we'll ask once here. You could modify this to ask per file later.
CPI = float(input("Enter mouse CPI (e.g. 800): "))
mm_per_count = 25.4 / CPI

# Process each uploaded file
for filename in filenames:
    print(f"Processing {filename}...")
    try:
        # Read the CSV file
        df = pd.read_csv(filename)

        # Perform calculations
        df['dx_mm'] = df.dx * mm_per_count
        df['dy_mm'] = df.dy * mm_per_count
        df['x'] = df.dx_mm.cumsum()
        df['y'] = df.dy_mm.cumsum()

        # Store the processed DataFrame
        dataframes[filename] = df
        print(f"Successfully processed {filename}")

    except Exception as e:
        print(f"Error processing {filename}: {e}")
        # Remove the filename from the list if processing fails
        filenames.remove(filename)
        if not filenames:
            print("No files were successfully processed.")
            # You might want to exit or handle this case appropriately
            # For now, we'll just stop if there are no valid files
            raise SystemExit("No valid files to plot.")


# Function to plot the current file's path
def plot_current_file():
    if not filenames:
        print("No files available to plot.")
        return

    current_filename = filenames[current_file_index]
    df = dataframes[current_filename]

    plt.figure(figsize=(6,6))
    plt.plot(df.x, df.y)
    plt.scatter(df.x.iloc[0], df.y.iloc[0], label='Start')
    plt.scatter(df.x.iloc[-1], df.y.iloc[-1], label='End')
    plt.title(f'OptiNav Taxi Path - {current_filename}')
    plt.axis('equal'); plt.grid(); plt.legend(); plt.show()

# Button click handlers
def on_next_button_clicked(b):
    global current_file_index
    if current_file_index < len(filenames) - 1:
        current_file_index += 1
        plot_current_file()
    else:
        print("Already at the last file.")

def on_prev_button_clicked(b):
    global current_file_index
    if current_file_index > 0:
        current_file_index -= 1
        plot_current_file()
    else:
        print("Already at the first file.")

# Create buttons
next_button = widgets.Button(description="Next File")
prev_button = widgets.Button(description="Previous File")

# Link button clicks to functions
next_button.on_click(on_next_button_clicked)
prev_button.on_click(on_prev_button_clicked)

# Display the buttons and the initial plot
display(prev_button, next_button)
plot_current_file()